In [19]:
from simalignmodified import SentenceAligner
import pandas as pd
import numpy as np
import json
import tqdm
import os
import truecase
from mosestokenizer import MosesDetokenizer
from mt_utils import (find_corpus, 
                      load_data, 
                      load_metadata, 
                      print_sys_level_correlation, 
                      print_seg_level_correlation,
                      df_append)

In [20]:
simaligner = SentenceAligner(matching_methods="i")

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
2021-12-20 14:51:22,768 - simalignmodified - INFO - Initialized the EmbeddingLoader with model: bert-base-multilingual-cased


In [29]:
dataset = find_corpus("WMT17")
    
pairs = []
for pair in dataset.items(): 
    pairs.append(pair)
reference_path, lp = pairs[0]
print(lp)
references = load_data(os.path.join( 'references/', reference_path))
src, tgt = lp.split('-')
source_path = reference_path.replace('ref', 'src')
source_path = source_path.split('.')[0] + '.' + src  
source = load_data(os.path.join('source', source_path))
all_meta_data = load_metadata(os.path.join('system-outputs', lp))
with MosesDetokenizer(src) as detokenize:        
    source = [detokenize(s.split(' ')) for s in source]         
with MosesDetokenizer(tgt) as detokenize:                
    references = [detokenize(s.split(' ')) for s in references]
trans = []

path, testset, lp, system = all_meta_data[0]

translations = load_data(path)        
num_samples = len(references)

with MosesDetokenizer(tgt) as detokenize:                    
    translations = [detokenize(s.split(' ')) for s in translations]

translations = [truecase.get_true_case(s) for s in translations]
scores = []
for i in tqdm(range(len(translations))):
    align , sim = simaligner.get_word_aligns(source[i], translations[i])
    length = len(align["itermax"])
    score = 0
    for p in align["itermax"]:
        score += sim[p[0]][p[1]]
    scores.append(score/length)


cs-en


TypeError: 'list' object cannot be interpreted as an integer

Osmadvacetiletý šéfkuchař nalezen mrtev v obchodě v San Francisku
- chef found dead in store in San Francisco


0.7040402624342177